# SGEN_Py Quick Start tutorial

## About
SGEN_Py (Stochastic Gene Expression in Neurons) is a Python module allowing computing the first and second moments of active gene, mRNA and protein counts everywhere in arbitrary neurons. It does so by solving a linear stochastic model of gene->mRNA->protein dynamics. The module can also be used to run Monte Carlo simulation of the underlying stochastic process, which can be used as a generative model of protein noise. However, the real strength of this software is its ability to obtain expectations and correlations of all variables in the model without expensive Monte Carlo simulation. 

In this tutorial we will
- Manually create ("grow") a simple neuron
- Compute stationary expectations and correlations of all variables: active gene counts, mRNA counts and protein counts
- Visualise the computed distributions
- Run Gillespie algoritm on this simple neuron
- Load a neuron from .swc file
- Visualise the neuron and the computed distributions


## Manual neuron
#### Importing dependencies

In [1]:
import sys
sys.path.append('../')

import SGEN_Py as sg
import numpy as np

### "Growing" a neuron
Below we create a Y-shaped neuron consisting of a cylindrical soma of radius and length of 20um and a dendrite with a single fork with every dendritic branch having length of 200um.
#### Creating a soma
In SGEN a neuron is a tree of compartments starting from the soma. 

In [3]:
# Create a soma with desired parameters and location
soma = sg.Soma("soma", length=20, x=0, y=0, z=0, radius=20)

#### Creating primary dendritic branch
When creating a compartment other than soma, we must pass its parent (i.e., the compartment it attaches to) to its constructor. A dendritic branch (a piece of dendrite without branching) can be represented as a list of dendritic segments used for discretisation. (The larger the number of dendritic segments, the closer the model is to a continuum, but the harder it is to compute.) Let's initialise the primary (directly attached to the soma) dendritic branch by attaching a dendritic segment to the soma.

In [1]:
# Define the length of a single dendritic branch
Dendrite_length = 200 #um
# Define the number of cylindrical segments to discretise dendritic branches
N_dendritic_segments = 15

In [4]:
# Initialise a primary dendritic branch
primary_branch = [sg.Dendritic_segment(parent=soma,
                                       name = "d_1-1",
                                       length = Dendrite_length/N_dendritic_segments)]

In the above code we created a list containing one dendritic segment with certain name and length attached to the soma. See help for all possible constructor arguments including rates of mRNA/protein dynamics.

Now let's create more dendritic segments to obtain the entire primary dendritic branch of length 200um.

In [5]:
# Adding dendritic segments to the primary dendritic branch
for i in np.arange(1,N_dendritic_segments):
    primary_branch.append(sg.Dendritic_segment(parent=primary_branch[i-1],
                                               name="d_1-" + str(i+1),
                                               length=Dendrite_length/N_dendritic_segments))

In the above code we created the primary dendritic branch as a list of short dendritic segments, where every next segment attaches to the previous one.

Now let's fork the dendrite and create two secondary dendritic branches attaching them to the last segment of the primary branch created above.

In [6]:
# Initialise 1st secondary dendritic branch
secondary_branch_1 = [sg.Dendritic_segment(parent=primary_branch[N_dendritic_segments-1],
                                           name="d_1_1-1",
                                           length=Dendrite_length/N_dendritic_segments,
                                           d_theta=30*np.pi/360,
                                           d_phi=0)]
# Adding dendritic segments to the 1st secondary dendritic branch
for i in np.arange(1,N_dendritic_segments):
    secondary_branch_1.append(sg.Dendritic_segment(parent=secondary_branch_1[i-1],
                                                   name="d_1_1-" + str(i+1),
                                                   length=Dendrite_length/N_dendritic_segments))

# Initialise 2nd secondary dendritic branch
secondary_branch_2 = [sg.Dendritic_segment(parent=primary_branch[N_dendritic_segments-1],
                                           name="d_1_1-1",
                                           length=Dendrite_length/N_dendritic_segments,
                                           d_theta=-30*np.pi/360,
                                           d_phi=0)]
# Adding dendritic segments to the 2nd secondary dendritic branch
for i in np.arange(1,N_dendritic_segments):
    secondary_branch_2.append(sg.Dendritic_segment(parent=secondary_branch_2[i-1],
                                                   name="d_1_1-" + str(i+1),
                                                   length=Dendrite_length/N_dendritic_segments,
                                                   radius=5*np.exp(-1/50*i)))

Note that we specified the parameters d_theta and d_phi, corresponding to the difference in the spherical coordinates ($\theta$ and $\phi$) compared to the parent compartment. These angular differences are set to zero by default, so if they are not specified, the dendrite grows in a straight line. 

Now let's "grow" a dendritic spine on our primary dendrite at roughly 1/3 of its length.

In [7]:
# Creating a dendritic spine on the primary branch
s_1_1 = sg.Spine(parent=primary_branch[N_dendritic_segments//3],
                 name="s_1_1",
                 length=10,
                 radius=1)

Now let's add five more spines at different locations and on different branches

In [8]:
# Adding more spines, some of which grow in opposite direction (d_theta=-np.pi/2)
s_1_2 = sg.Spine(parent=primary_branch[2*N_dendritic_segments//3],
                 name="s_1_2",
                 length=10,
                 radius=1,
                 d_theta=-np.pi/2)
s_11_1 = sg.Spine(parent=secondary_branch_1[N_dendritic_segments//3],
                  name = "s_11_1",
                  length=10,
                  radius=1)
s_11_2 = sg.Spine(parent=secondary_branch_1[2*N_dendritic_segments//3],
                  name = "s_11_2",
                  length=10,
                  radius=1,
                  d_theta=-np.pi/2)
s_12_1 = sg.Spine(parent=secondary_branch_2[N_dendritic_segments//3],
                  name="s_12_1",
                  length=10,
                  radius=1,
                  d_theta=-np.pi/2)
s_12_2 = sg.Spine(parent=secondary_branch_2[2*N_dendritic_segments//3],
                  name="s_12_2",
                  length=10,
                  radius=1)

### Initialising a neuron
Now that the tree of compartments is ready, we can initialise the instance of class Neuron that provides the main functionality of the library. For that we pass the soma to the constructor of the Neuron class.

In [9]:
neuron = sg.Neuron(soma, "Test_neuron")

### Visualisation

Now let's visualise our neuron.

In [10]:
neuron.draw_3d()

## Stationary gene-mRNA-protein moments

The expectations of all variables in the model can be computed as follows.

In [11]:
# Computing expectations of all variables in the model
expectations = neuron.expected_counts(dict_return=True)
print(expectations)

Expectations is a dictionary with the following keys.

In [12]:
expectations.keys()

If run with dict_return=True, neuron.expected_counts(dict_return=True) returns dictionaries for 'mRNA' and 'prot', whose keys correspond to compartment names. (Since all genes reside in the soma, there is no need to return the dictionary for the expected number of active genes, which can simply be accessed by the 'gene' key.) So the expected mRNA/protein counts in different compartments can be obtaines as follows.

In [13]:
print('Expected number of active genes:', expectations['gene'])
print('Expected mRNA counts in the soma:', expectations['mRNA']['soma'])
print('Expected mRNA counts in d_1_1-7:', expectations['mRNA']['d_1_1-7'])
print('Expected protein counts in d_1_1-15:', expectations['prot']['d_1_1-15'])

#### Visualisation of concentrations
Expected concentrations are obtained by dividing the expected molecule counts by the volumes of the corresponding volumes as follows.

In [14]:
# Computing protein concentrations
concentrations = neuron.expected_counts()['prot']/neuron.volumes()
print(concentrations)

Note that in the above code we call neuron.expected_counts() with default dict_return=False, so that neuron.expected_counts()['prot'] returns a numpy array that can be divided by a numpy array of volumes of all compartments ordered in the same order, neuron.volumes().
Now we can visualise protein concentrations as follows.

In [15]:
neuron.draw_3d(concentrations)

### Stationary correlations
Now we compute stationary correlations (2nd moments) of all variables in the model defined as $\langle A, B \rangle$ for variables $A$ and $B$, where $\langle . \rangle$ stands for expectation (ensemble average).

In [16]:
correlations = neuron.correlations()

In our model, there are six types of correlations: gene-gene, gene-mRNA, mRNA-mRNA, gene-protein, mRNA-protein, protein-protein. Therefore, neuron.correlations() returns stationary correlations as a dictionary with the following keys.

Protein-protein correlators turn out to be computationally hardest to compute and the exact method involving matrix inversion scales as $\mathcal O(N^4)$ in memory consumption and $\mathcal O(N^6)$ in time, where $N$ is the number of compartments.

In [17]:
print(correlations.keys())

Let's look at some of the correlation matrices.

In [34]:
# gene-gene correlation is just a single number
print('Active gene count variance:', correlations['gene-gene'])
# gene-mRNA and gene-protein correlations are vectors
print('Correlations of active gene count with mRNA counts:\n', correlations['gene-mRNA'])
# mRNA-mRNA, mRNA-protein and protein-protein correlations are matrices of different sizes
print('Correlations of protein counts with protein counts:\n', correlations['prot-prot'])
# Note that mRNA-protein correlation matrix is not square in the presence of spines.
# This comes from the assumption that spines don't have mRNAs (which we may change). 
print('Shape of correlations of mRNA counts with protein counts:\n', correlations['mRNA-prot'].shape)